In [ ]:
pip install tmdbsimple


In [1]:
import pandas as pd

# Preparation

In [ ]:
df = pd.read_json('/Users/beebo/code/rmelbardis/ObjectivelyFunny/raw_data/filtered_scraps_from_the_loft.json')
df.head()

In [ ]:
title = df['title']

In [ ]:
df.sort_values('title')[:10]

# Example of using tmdsimple

In [2]:
import tmdbsimple as tmdb
import os
from dotenv import load_dotenv, find_dotenv


In [3]:
# point to api key in the .env file 
env_path = find_dotenv() 
load_dotenv(env_path)
tmdb.API_KEY = os.getenv('TMDB_KEY')

## Step 1: searching for the actor's id

In [4]:
name = 'ADEL KARAM'

In [5]:
search = tmdb.Search()
response = search.person(query=name)
print([s['id'] for s in search.results])

[53445]


## Step 2: using the id to search for info

In [6]:
identity = tmdb.People(53445)
response = identity.info()
print(identity.birthday, identity.gender, identity.place_of_birth)

1972-08-20 2 Beirut, Lebanon


handling multiple search results (usually the first one, but need to check)

In [7]:
response = search.person(query='ALI WONG')
[s['id'] for s in search.results]

[591835, 2946658, 3156949]

In [8]:
identity = tmdb.People(591835)
response = identity.info()
print(identity.birthday, identity.gender, identity.place_of_birth)

1982-04-19 1  San Francisco, California, USA


## Putting things together

In [9]:
def extraction(names):
    '''
    Taking in a list of actor/actress' names
    Returning a dictionary containing their birthday, gender and birthplace
    '''
    birthday = []
    gender = []
    birthplace = []
    
    search = tmdb.Search()
    for name in names:
        response = search.person(query=name)
        id_list = [s['id'] for s in search.results]
        # if there is no id result
        if len(id_list) < 1:
            print(f'No {name} found. Please check the name.')
            birthday.append(None)
            gender.append(None)
            birthplace.append(None)
        # if there is more than one result
        else:
            if len(id_list) > 1:
                print('Multiple people found.')
                print(id_list)
            first_id = id_list[0]
            identity = tmdb.People(first_id)
            response = identity.info()

            birthday.append(identity.birthday)
            gender.append(identity.gender)
            birthplace.append(identity.place_of_birth)
    return {'name': names, 'birthday': birthday, 'gender': gender, 'birthplace': birthplace}

In [10]:
new_dict = extraction(['AMY SCHUMER', 'goosdsds', 'dsfsfdsdfs', 'AZIZ ANSARI'])
pd.DataFrame.from_dict(new_dict)

No goosdsds found. Please check the name.
No dsfsfdsdfs found. Please check the name.


,name,birthday,gender,birthplace
0,AMY SCHUMER,1981-06-01,1.0,"New York, New York, USA"
1,goosdsds,None,NaN,None
2,dsfsfdsdfs,None,NaN,None
3,AZIZ ANSARI,1983-02-23,2.0,"Columbia, South Carolina, USA"
